<a href="https://colab.research.google.com/github/anudaweerasinghe/small-coder/blob/main/tiny_llama_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/anudaweerasinghe/small-coder.git
%cd /content/small-coder

fatal: destination path 'small-coder' already exists and is not an empty directory.
/content/small-coder


In [2]:
! pip install transformers datasets

In [15]:
from datasets import load_dataset

dataset = load_dataset("iamtarun/python_code_instructions_18k_alpaca")

Generating train split:   0%|          | 0/18612 [00:00<?, ? examples/s]

In [16]:
dataset['train'][1]

{'instruction': 'Generate a Python code for crawling a website for a specific type of data.',
 'input': 'website: www.example.com \ndata to crawl: phone numbers',
 'output': "import requests\nimport re\n\ndef crawl_website_for_phone_numbers(website):\n    response = requests.get(website)\n    phone_numbers = re.findall('\\d{3}-\\d{3}-\\d{4}', response.text)\n    return phone_numbers\n    \nif __name__ == '__main__':\n    print(crawl_website_for_phone_numbers('www.example.com'))",
 'prompt': "Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nGenerate a Python code for crawling a website for a specific type of data.\n\n### Input:\nwebsite: www.example.com \ndata to crawl: phone numbers\n\n### Output:\nimport requests\nimport re\n\ndef crawl_website_for_phone_numbers(website):\n    response = requests.get(website)\n    phone_numbers = re.findall('\\d{3}-\\d{3}-\\d{4}', response.text)\n    return phone_numbers\n  

In [21]:
def format_instruction(sample):
	return f"""### Instruction:
Below is an instruction that describes a task. Write a response that appropriately completes the request:

### Task:
{sample['instruction']}

### Input:
{sample['input']}

### Response:
{sample['output']}
"""

In [29]:
import torch

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments

In [ ]:
model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

In [33]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_id)

In [34]:
# IDK ABOUT THIS TBH
def tokenize_function(examples):
    return tokenizer(examples["prompt"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/18612 [00:00<?, ? examples/s]

In [35]:
from transformers import AutoModel

model = AutoModel.from_pretrained(model_id)

In [36]:
!pip install accelerate>=0.20.3 transformers

In [37]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

In [38]:
!pip install evaluate

In [39]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [40]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [41]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch", per_device_train_batch_size=8)

In [43]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    compute_metrics=compute_metrics
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [44]:
trainer.train()

ValueError: The model did not return a loss from the inputs, only the following keys: last_hidden_state,past_key_values. For reference, the inputs it received are input_ids,attention_mask.